In [13]:
import pandas as pd
import numpy as np

In [18]:
# Load the Excel file
file_path = 'output.xlsx'
xls = pd.ExcelFile(file_path)

In [21]:
# Read each sheet into a DataFrame without using the first column as an index
energy_df = pd.read_excel(xls, 'Energy')
soc_df = pd.read_excel(xls, 'SOC')
power_df = pd.read_excel(xls, 'Power')
charger_enabled_df = pd.read_excel(xls, 'Charger Enabled')
charger_assigned_df = pd.read_excel(xls, 'Charger Assigned')

# Drop any Unnamed columns if they still exist
energy_df = energy_df.loc[:, ~energy_df.columns.str.contains('^Unnamed')]
soc_df = soc_df.loc[:, ~soc_df.columns.str.contains('^Unnamed')]
power_df = power_df.loc[:, ~power_df.columns.str.contains('^Unnamed')]
charger_enabled_df = charger_enabled_df.loc[:, ~charger_enabled_df.columns.str.contains('^Unnamed')]
charger_assigned_df = charger_assigned_df.loc[:, ~charger_assigned_df.columns.str.contains('^Unnamed')]

# Calculate total energy consumption (fleet and per bus)
total_energy_consumption_per_bus = energy_df.max() - energy_df.min()
total_energy_consumption_fleet = total_energy_consumption_per_bus.sum()

# Calculate total charging time (fleet and per bus) based on energy increase
charging_step_value = 15  # in minutes
energy_diff = energy_df.diff()
charging_time_per_bus = (energy_diff > 0).sum() * (charging_step_value / 60)  # converting minutes to hours
total_charging_time_fleet = charging_time_per_bus.sum()

# Calculate average energy consumption (fleet)
average_energy_consumption_fleet = total_energy_consumption_fleet / len(total_energy_consumption_per_bus)

# Calculate average charging time (fleet)
average_charging_time_fleet = total_charging_time_fleet / len(charging_time_per_bus) if len(charging_time_per_bus) > 0 else 0

# Calculate maximum and minimum SOC (per bus)
max_soc_per_bus = soc_df.max()
min_soc_per_bus = soc_df.min()

# Calculate average SOC (per bus)
average_soc_per_bus = soc_df.mean()

# Calculate charging window (per bus)
charging_window_per_bus = charger_assigned_df[charger_assigned_df['Charging_Status'] == 1].groupby('Bus')['Time'].agg(['min', 'max'])

# Calculate charger number (per bus)
charger_number_per_bus = charger_assigned_df.groupby('Bus')['Charger'].max()

# Calculate state percentages (idle, running, charging)
# Determine state for each timestep and bus
states = pd.DataFrame(index=energy_diff.index, columns=energy_diff.columns)
states[energy_diff > 0] = 'Charging'
states[energy_diff < 0] = 'Running'
states[energy_diff == 0] = 'Idle'

# Calculate state percentages for each bus
state_percentage_per_bus = states.apply(lambda col: col.value_counts(normalize=True) * 100).fillna(0)

# Calculate fleet state percentages
fleet_states = states.apply(pd.Series.value_counts, axis=1).fillna(0)
fleet_states = fleet_states.div(fleet_states.sum(axis=1), axis=0) * 100
fleet_idle_percentage = fleet_states['Idle'].mean() if 'Idle' in fleet_states.columns else 0
fleet_running_percentage = fleet_states['Running'].mean() if 'Running' in fleet_states.columns else 0
fleet_charging_percentage = fleet_states['Charging'].mean() if 'Charging' in fleet_states.columns else 0

# Display the results
print(f"Total Energy Consumption (Fleet): {total_energy_consumption_fleet} kWh")
print(f"Total Energy Consumption (Per Bus):")
print(total_energy_consumption_per_bus)

print(f"Total Charging Time (Fleet): {total_charging_time_fleet} hours")
print(f"Total Charging Time (Per Bus):")
print(charging_time_per_bus)

print(f"Average Energy Consumption (Fleet): {average_energy_consumption_fleet} kWh")
print(f"Average Charging Time (Fleet): {average_charging_time_fleet} hours")

print(f"Maximum SOC (Per Bus):")
print(max_soc_per_bus)
print(f"Minimum SOC (Per Bus):")
print(min_soc_per_bus)
print(f"Average SOC (Per Bus):")
print(average_soc_per_bus)

print(f"Charging Window (Per Bus):")
print(charging_window_per_bus)

print(f"Charger Number (Per Bus):")
print(charger_number_per_bus)

print(f"State Percentages (Fleet): Idle: {fleet_idle_percentage:.2f}%, Running: {fleet_running_percentage:.2f}%, Charging: {fleet_charging_percentage:.2f}%")
print(f"State Percentages (Per Bus):")
print(state_percentage_per_bus)


Total Energy Consumption (Fleet): 2925.0 kWh
Total Energy Consumption (Per Bus):
bus 1     135.0
bus 2     157.5
bus 3     135.0
bus 4     157.5
bus 5     157.5
bus 6     135.0
bus 7     135.0
bus 8     135.0
bus 9     157.5
bus 10    135.0
bus 11    157.5
bus 12    157.5
bus 13    157.5
bus 14    135.0
bus 15    157.5
bus 16    157.5
bus 17    135.0
bus 18    135.0
bus 19    157.5
bus 20    135.0
dtype: float64
Total Charging Time (Fleet): 32.5 hours
Total Charging Time (Per Bus):
bus 1     1.50
bus 2     1.75
bus 3     1.50
bus 4     1.75
bus 5     1.75
bus 6     1.50
bus 7     1.50
bus 8     1.50
bus 9     1.75
bus 10    1.50
bus 11    1.75
bus 12    1.75
bus 13    1.75
bus 14    1.50
bus 15    1.75
bus 16    1.75
bus 17    1.50
bus 18    1.50
bus 19    1.75
bus 20    1.50
dtype: float64
Average Energy Consumption (Fleet): 146.25 kWh
Average Charging Time (Fleet): 1.625 hours
Maximum SOC (Per Bus):
bus 1     87.50
bus 2     98.75
bus 3     87.50
bus 4     98.75
bus 5     98.75
bus 6

In [26]:
# Prepare data for Excel output
results = {
    'Total Energy Consumption (Fleet)': [total_energy_consumption_fleet],
    'Total Charging Time (Fleet)': [total_charging_time_fleet],
    'Average Energy Consumption (Fleet)': [average_energy_consumption_fleet],
    'Average Charging Time (Fleet)': [average_charging_time_fleet],
    'Fleet Idle Percentage': [fleet_idle_percentage],
    'Fleet Running Percentage': [fleet_running_percentage],
    'Fleet Charging Percentage': [fleet_charging_percentage],
}

# Create an Excel writer
output_path = 'analytics_output.xlsx'
with pd.ExcelWriter(output_path) as writer:
    # Write individual DataFrames to separate sheets
    total_energy_consumption_per_bus.to_frame(name='Energy Consumption Per Bus (kWh)').to_excel(writer, sheet_name='Energy Consumption Per Bus')
    charging_time_per_bus.to_frame(name='Charging Time Per Bus (hour)').to_excel(writer, sheet_name='Charging Time Per Bus')
    max_soc_per_bus.to_frame(name='Max SOC Per Bus (%)').to_excel(writer, sheet_name='Max SOC Per Bus')
    min_soc_per_bus.to_frame(name='Min SOC Per Bus (%)').to_excel(writer, sheet_name='Min SOC Per Bus')
    average_soc_per_bus.to_frame(name='Average SOC Per Bus (%)').to_excel(writer, sheet_name='Avg SOC Per Bus')
    charging_window_per_bus.to_excel(writer, sheet_name='Charging Window Per Bus')
    charger_number_per_bus.to_frame(name='Charger Number Per Bus').to_excel(writer, sheet_name='Charger Number Per Bus')
    state_percentage_per_bus.to_excel(writer, sheet_name='State Percentage Per Bus (%)')
    
    # Write summary results to a sheet
    pd.DataFrame(results).to_excel(writer, sheet_name='Fleet Summary')

print(f"Analytics results saved to {output_path}")

Analytics results saved to analytics_output.xlsx
